In [12]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
# Read all models
import sys
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

METHOD = "hmm_pomegranate"
from hmm_logic_methods import  load_model
from hmm_visualization_methods import *
from training_parameters import *

import training_parameters
from data_reading_methods import *
from pomegranate.io import BatchedDataGenerator, SequenceGenerator
import subprocess

import pandas as pd

In [13]:
import os
import glob
import numpy as np
from collections import defaultdict

# Your desired alleles (in IMGT format)
desired_alleles = [
    'HLA-DRB1*03:01', 'HLA-DRB1*07:01', 'HLA-DRB1*11:01', 'HLA-DRB1*12:01',
    'HLA-DRB1*15:01', 'HLA-DRB3*01:01', 'HLA-DRB3*02:02', 'HLA-DRB4*01:01'
]

# Mapping NetMHCIIpan allele format to IMGT format
allele_map = {
    'DRB1_0301': 'HLA-DRB1*03:01',
    'DRB1_0701': 'HLA-DRB1*07:01',
    'DRB1_1101': 'HLA-DRB1*11:01',
    'DRB1_1201': 'HLA-DRB1*12:01',
    'DRB1_1501': 'HLA-DRB1*15:01',
    'DRB3_0101': 'HLA-DRB3*01:01',
    'DRB3_0202': 'HLA-DRB3*02:02',
    'DRB4_0101': 'HLA-DRB4*01:01'
}

# Path to your folder
folder_path = r"C:\Projects\grandmaster\notebooks\MHC_predictor\validation_data\NetMHCIIpan_train"
file_pattern = os.path.join(folder_path, "test_BA*.txt")
files = sorted(glob.glob(file_pattern))

# Output dictionaries
peptides_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
scores_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

# Process each file
for split_index, file_path in enumerate(files):
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            peptide, score, allele_raw, _ = line.split('\t')
            score = float(score)

            allele_clean = allele_raw.replace('-', '')  # e.g., HLA-DRB10301
            allele = allele_map.get(allele_clean)
            if allele is None or allele not in desired_alleles:
                continue  # Skip unwanted alleles

            length = len(peptide)
            peptides_dict[allele][split_index][length].append(peptide)
            scores_dict[allele][split_index][length].append(score)

# Convert lists to numpy arrays
for allele in peptides_dict:
    for split in peptides_dict[allele]:
        for length in peptides_dict[allele][split]:
            peptides_dict[allele][split][length] = np.array(peptides_dict[allele][split][length])
            scores_dict[allele][split][length] = np.array(scores_dict[allele][split][length])


In [20]:
base_dir = r"experiment_results\reordered_models"
base_models = [f"base_model_{i}" for i in range(4,5)]
new_models = [f"second_model_{i}" for i in range(4,10)]

experiments = base_models #+ new_models


In [5]:

#read and prepare data
def get_train_test_data(experiment_params :ExperimentParams, type_of_peptides= "binders"):
    data_params : DataScenarioParams = experiment_params.data_scenario_params
    data_scenario = data_params.data_scenario
    DATA_PATH = data_params.input_data_path
    TOTAL_SPLITS = data_params.splits_to_read
    print(f"Will read files from the folder {DATA_PATH}")
    assert data_scenario in ["IEDB_preprocessed", "simulated", "simulated_preprocessed", 'MixMHCpred']
    additional_return = list()
    if isinstance(data_params, PreprocessedIEDBDataParams):
        ALLELES = get_available_alleles(DATA_PATH)
        per_allele_per_kfold_per_length_binders_train = read_data(DATA_PATH,ALLELES, "train", type_of_peptides)
        per_allele_per_kfold_per_length_binders_test = read_data(DATA_PATH,ALLELES, "test", type_of_peptides)
        sample_allele = list(per_allele_per_kfold_per_length_binders_train.keys())[0]
        per_allele_df = join_dicts(per_allele_per_kfold_per_length_binders_train)
        for allele_name in ALLELES:
            per_allele_df[allele_name]['allele'] = allele_name
        assert len(per_allele_per_kfold_per_length_binders_train[sample_allele].keys()) >= TOTAL_SPLITS # check number of splits
        additional_return.append(per_allele_df)
    elif isinstance(data_params, SimulatedPreprocessedDataParams):
        ALLELES = get_available_alleles(DATA_PATH, do_not_parse_alleles=True)
        per_allele_per_kfold_per_length_binders_train = read_data(DATA_PATH,ALLELES, "train", type_of_peptides, do_not_parse_alleles=True)
        per_allele_per_kfold_per_length_binders_test = read_data(DATA_PATH,ALLELES, "test", type_of_peptides,do_not_parse_alleles=True)
        sample_allele = list(per_allele_per_kfold_per_length_binders_train.keys())[0]
        per_allele_df = join_dicts(per_allele_per_kfold_per_length_binders_train)
        for allele_name in ALLELES:
            per_allele_df[allele_name]['allele'] = allele_name
        assert len(per_allele_per_kfold_per_length_binders_train[sample_allele].keys()) >= TOTAL_SPLITS # check number of splits
        additional_return.append(per_allele_df)
    elif isinstance(data_params, SimulatedDataParams):
        simulated_exact_file = data_params.simulated_exact_file_name
        dummy_allele_name = data_params.dummy_allele_name
        simulated_scenario = data_params.simulated_scenario
        SIMULATED_DATA_PATH = f"{DATA_PATH}/{simulated_scenario}/{simulated_exact_file}"
        ALLELES = [dummy_allele_name]
        per_allele_df = dict()
        # For now just read the same data multiple times for alleles/splits
        for allele_name in ALLELES:
            allele_df = pd.read_csv(SIMULATED_DATA_PATH, sep=";")
            list_dfs = [allele_df.copy() for i in range(TOTAL_SPLITS)]
            for split_num, split_df in enumerate(list_dfs):
                split_df['split'] = split_num
                split_df['allele_name'] = allele_name
                result_allele_df = pd.concat(list_dfs)
            per_allele_df[allele_name] = result_allele_df
            result_allele_df['length'] = split_df.peptide.str.len()
            TARGET_LENGTHS = list(split_df['length'].unique())
        # split data into dicts
        per_allele_per_kfold_per_length_binders_train = split_to_dicts(per_allele_df,
                                                                  ALLELES=ALLELES,
                                                                  TARGET_LENGTHS=TARGET_LENGTHS,
                                                                  TOTAL_SPLITS=np.arange(TOTAL_SPLITS))
        per_allele_per_kfold_per_length_binders_test =  split_to_dicts(per_allele_df,
                                                                  ALLELES=ALLELES,
                                                                  TARGET_LENGTHS=TARGET_LENGTHS,
                                                                  TOTAL_SPLITS=np.arange(TOTAL_SPLITS))
        additional_return.append(per_allele_df)
    elif isinstance(data_params, MixMHCpredDataParams):
        mixture_name = data_params.mixmhc_mixture_name
        dummy_allele_name = data_params.dummy_allele_name
        df = pd.read_csv(DATA_PATH, sep=';')
        print(df.columns)
        df = df.loc[
            df.Peptide.str.match("^[ACDEFGHIKLMNPQRSTVWY]+$")
        ]
        print("Total table length", len(df))
        #Filter out selected mixture
        df = df.loc[df.Sample_IDs.str.split(', ').apply(lambda x: mixture_name in x),]
        print("Filtered for given mixture", len(df))
        sample_data = pd.DataFrame(
            {"peptide": df.Peptide.values,
             "old_sample_id": df.Sample_IDs,
             "sample_id": mixture_name,
             "mixmhc_predicted_mixed_alleles": df.Allele.values})

        list_dfs = [sample_data.copy() for i in range(TOTAL_SPLITS)]
        per_allele_df = dict()
        allele_name = "d_" + dummy_allele_name
        ALLELES = [allele_name]
        for allele_name in ALLELES:
            for split_num, split_df in enumerate(list_dfs):
                split_df['split'] = split_num
                split_df['allele'] = allele_name
                split_df['length'] = split_df.peptide.str.len()
            result_allele_df = pd.concat(list_dfs)
            result_allele_df = result_allele_df.drop_duplicates(subset=['peptide'])
            TARGET_LENGTHS = list(split_df['length'].unique())
            per_allele_df[allele_name] = result_allele_df
        per_allele_per_kfold_per_length_binders_train = split_to_dicts(per_allele_df,
                                                                  ALLELES=ALLELES,
                                                                  TARGET_LENGTHS=TARGET_LENGTHS,
                                                                  TOTAL_SPLITS=np.arange(TOTAL_SPLITS))
        per_allele_per_kfold_per_length_binders_test = split_to_dicts(per_allele_df,
                                                                  ALLELES=ALLELES,
                                                                  TARGET_LENGTHS=TARGET_LENGTHS,
                                                                  TOTAL_SPLITS=np.arange(TOTAL_SPLITS))
        additional_return.append(per_allele_df)
        additional_return.append(df)
    return per_allele_per_kfold_per_length_binders_train,  per_allele_per_kfold_per_length_binders_test, additional_return

from pomegranate.io import BatchedDataGenerator, SequenceGenerator
def create_char_arrays(peptide_sequences):
    return np.array([[char for char in peptide] for peptide in peptide_sequences], dtype=object)

def prepare_split_data_separeted_length(per_length_data, per_length_weights, per_length_test_data, target_lengths):
    binders_array = np.array([per_length_data[length][i] for length in target_lengths
                              for i in range(len(per_length_data[length]))], dtype=object)
    weights_array = np.array([per_length_weights[length][i] for length in target_lengths
                               for i in range(len(per_length_weights[length]))], dtype=object)
    binders_test_array = np.array([per_length_test_data[length][i] for length in target_lengths
                                   for i in range(len(per_length_test_data[length]))], dtype=object)
    return binders_array, weights_array, binders_test_array




def read_all_models(models_dir, history=True):
    parsed_alleles = os.listdir(models_dir)
    print(f"Found {parsed_alleles}")    
    per_allele_per_split_per_run_model = dict()
    per_allele_per_split_per_run_history = dict() if history else None
    
    for parsed_allele in parsed_alleles:
        per_allele_per_split_per_run_model[parsed_allele] = dict()
        if history:
            per_allele_per_split_per_run_history[parsed_allele] = dict()
        
        allele_dir = f"{models_dir}/{parsed_allele}"
        for split_string in os.listdir(allele_dir):
            per_allele_per_split_per_run_model[parsed_allele][split_string] = dict()
            if history:
                per_allele_per_split_per_run_history[parsed_allele][split_string] = dict()
            
            split_dir = f"{allele_dir}/{split_string}/"
            for run_string in os.listdir(split_dir):
                run_dir = f"{split_dir}{run_string}"
                run_index = run_string.split("_")[0]
                model = load_model(split_dir, run_string)
                per_allele_per_split_per_run_model[parsed_allele][split_string][run_index] = model
                
                if history:
                    history_data = pd.read_csv(os.path.join(run_dir, "history.csv"), index_col=False, header=0)
                    per_allele_per_split_per_run_history[parsed_allele][split_string][run_index] = history_data

    if history:
        return per_allele_per_split_per_run_model, per_allele_per_split_per_run_history
    else:
        return per_allele_per_split_per_run_model


def score_binder(peptide, model_b, model_nb): 

    logp_1, _ = model_b.viterbi(np.array(list(peptide)))
    logp_1 = logp_1/len(peptide)
    logp_0, _ = model_nb.viterbi(np.array(list(peptide)))
    logp_0 = logp_0/len(peptide)
    score = np.exp(logp_1)/(np.exp(logp_1)+np.exp(logp_0))
    return score


import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def get_median_scores_for_top_models(per_name_models, peptides, models_fraction=0.25):
    runs = list(per_name_models.keys())
    num_models = max(int(len(runs) * models_fraction), 1)
    scores = np.zeros((len(peptides), num_models))
    
    for i, run in enumerate(runs[:num_models]):
        model = per_name_models[run]
        scores[:, i] = [model.log_probability(peptide) / len(peptide) for peptide in peptides]
    
    return np.median(scores, axis=1)

def prepare_peptides_data(test_data_b, test_data_nb, allele, split):
    peptides = []
    y_true = []
    
    for length, peptides_list in test_data_b.get(allele, {}).get(split, {}).items():
        peptides.extend(peptides_list)
        y_true.extend([1] * len(peptides_list))
    
    for length, peptides_list in test_data_nb.get(allele, {}).get(split, {}).items():
        peptides.extend(peptides_list)
        y_true.extend([0] * len(peptides_list))
    
    return peptides, y_true

def calculate_scores_for_allele_split(runs_b, runs_nb, peptides):
    if not peptides:
        return np.array([])
    
    logp_1 = get_median_scores_for_top_models(runs_b, peptides)
    logp_0 = get_median_scores_for_top_models(runs_nb, peptides)
    
    scores_adj = np.exp(logp_1) / (np.exp(logp_1) + np.exp(logp_0))
    return scores_adj

def evaluate_models(test_data_b, test_data_nb, per_name_models_b, per_name_models_nb):
    results, results_scores, y_true_all = {}, {}, {}
    
    for allele in set(test_data_b.keys()).union(test_data_nb.keys()):
        results[allele] = {}
        results_scores[allele] = {}
        y_true_all[allele] = {}
        allele_formated = allele.replace('-', '_').replace('*', '_').replace(':', '_').replace('/', '_')

        for split in set(test_data_b.get(allele, {}).keys()).union(test_data_nb.get(allele, {}).keys()):
            split_formated = f"s{split}"
            peptides, y_true = prepare_peptides_data(test_data_b, test_data_nb, allele, split)
            
            runs_b = per_name_models_b.get(allele_formated, {}).get(split_formated, {})
            runs_nb = per_name_models_nb.get(allele_formated, {}).get(split_formated, {})
            if not runs_b or not runs_nb:
                continue

            scores_adj = calculate_scores_for_allele_split(runs_b, runs_nb, peptides)
            
            results_scores[allele][split] = scores_adj.tolist()
            y_true_all[allele][split] = y_true
            
            if len(y_true) == len(scores_adj):
                results[allele][split] = {
                    "accuracy": accuracy_score(y_true, np.round(scores_adj)),
                    "precision": precision_score(y_true, np.round(scores_adj)),
                    "recall": recall_score(y_true, np.round(scores_adj)),
                    "f1": f1_score(y_true, np.round(scores_adj)),
                    "auc": roc_auc_score(y_true, scores_adj)
                }
    
    return results, results_scores, y_true_all

def save_json(data, filename, target_path):
    path = os.path.join(target_path, filename)
    if not os.path.exists(target_path):
        os.makedirs(target_path)
    with open(path, 'w') as file:
        json.dump(data, file, indent=4)

#preapare data to test 
hmm_params = training_parameters.ExperimentParams(experiment_name="reordered_models")
hmm_params.model_training_params = training_parameters.SimpleModelClassIIParamsMixMHC()
hmm_params.data_scenario_params.input_data_path = r'C:\Projects\grandmaster\notebooks\alleles_data\simple_model_enrichment\per_length_per_kfold_split'
hmm_params.data_scenario_params.splits_to_read = 4
model_training_params = hmm_params.model_training_params
per_allele_per_kfold_per_length_binders_train, \
per_allele_per_kfold_per_length_binders_test, additional_data = get_train_test_data(hmm_params)
per_allele_per_kfold_per_length_non_binders_train, \
per_allele_per_kfold_per_length_non_binders_test, additional_data = get_train_test_data(hmm_params, "nonbinders")
PARSED_ALLELES = list(per_allele_per_kfold_per_length_binders_train.keys())
PARSED_ALLELES_NB = list(per_allele_per_kfold_per_length_non_binders_train.keys())

model_training_params.lengths_to_use = [12, 13, 14, 15, 16, 17, 18, 19, 20]
df = additional_data[0][list(additional_data[0].keys())[0]]

current_mix = ['HLA-DRB1*03:01', 'HLA-DRB1*07:01', 
               'HLA-DRB1*12:01', 'HLA-DRB1*11:01', 
               'HLA-DRB1*15:01', 'HLA-DRB3*01:01', 
               'HLA-DRB3*02:02', 'HLA-DRB4*01:01']
model_training_params: training_parameters.ModelTrainingParams = hmm_params.model_training_params
model_training_params.alleles_to_use = [ item for item in current_mix]

#model_training_params.alleles_to_use = [ item for item in PARSED_ALLELES if item in [current_mix]]
t = remove_unused_lengths(per_allele_per_kfold_per_length_binders_train, experiment_params=hmm_params)
per_allele_per_kfold_per_length_binders_train = remove_unused_lengths(per_allele_per_kfold_per_length_binders_train, experiment_params=hmm_params)
per_allele_per_kfold_per_length_binders_test = remove_unused_lengths(per_allele_per_kfold_per_length_binders_test,experiment_params=hmm_params)

# transform to properties if needed and join multiple alleles
train_data_b, test_data_b, NEW_ALLELES, \
old_train_data_b, old_test_data_b, OLD_ALLELES = transform_data_to_properties_and_join_alleles(
    per_allele_per_kfold_per_length_binders_train,
    per_allele_per_kfold_per_length_binders_test,
    hmm_params.model_training_params
)
# Calculate weights for the training data based on peptide couns/lengths for unmerged data
#train_data_weigths_b = calculate_weights_based_on_length_counts(old_train_data_b, experiment_params=hmm_params)
per_allele_per_kfold_per_length_non_binders_train = remove_unused_lengths(per_allele_per_kfold_per_length_non_binders_train, experiment_params=hmm_params)
per_allele_per_kfold_per_length_non_binders_test = remove_unused_lengths(per_allele_per_kfold_per_length_non_binders_test,experiment_params=hmm_params)

# transform to properties if needed and join multiple alleles
train_data_nb, test_data_nb, NEW_ALLELES, \
old_train_data_nb, old_test_data_nb, OLD_ALLELES = transform_data_to_properties_and_join_alleles(
    per_allele_per_kfold_per_length_non_binders_train,
    per_allele_per_kfold_per_length_non_binders_test,
    hmm_params.model_training_params
)
# Calculate weights for the training data based on peptide couns/lengths for unmerged data
#train_data_weigths_nb = calculate_weights_based_on_length_counts(old_train_data_nb, experiment_params=hmm_params)
train_data_b, test_data_b, NEW_ALLELES, \
old_train_data_b, old_test_data_b, OLD_ALLELES = transform_data_to_properties_and_join_alleles(
    per_allele_per_kfold_per_length_binders_train,
    per_allele_per_kfold_per_length_binders_test,
    hmm_params.model_training_params
)
train_data_nb, test_data_nb, NEW_ALLELES, \
old_train_data_nb, old_test_data_nb, OLD_ALLELES = transform_data_to_properties_and_join_alleles(
    per_allele_per_kfold_per_length_non_binders_train,
    per_allele_per_kfold_per_length_non_binders_test,
    hmm_params.model_training_params
)



Will read files from the folder C:\Projects\grandmaster\notebooks\alleles_data\simple_model_enrichment\per_length_per_kfold_split
Will read files from the folder C:\Projects\grandmaster\notebooks\alleles_data\simple_model_enrichment\per_length_per_kfold_split
HLA-DRB1*03:01
Length 12: HLA-DRB1*03:01 - 135 
Length 13: HLA-DRB1*03:01 - 1555 
Length 14: HLA-DRB1*03:01 - 508 
Length 15: HLA-DRB1*03:01 - 1240 
Length 16: HLA-DRB1*03:01 - 794 
Length 17: HLA-DRB1*03:01 - 651 
Length 18: HLA-DRB1*03:01 - 460 
Length 19: HLA-DRB1*03:01 - 282 
Length 20: HLA-DRB1*03:01 - 408 
HLA-DRB1*07:01
Length 12: HLA-DRB1*07:01 - 163 
Length 13: HLA-DRB1*07:01 - 502 
Length 14: HLA-DRB1*07:01 - 732 
Length 15: HLA-DRB1*07:01 - 3366 
Length 16: HLA-DRB1*07:01 - 1000 
Length 17: HLA-DRB1*07:01 - 654 
Length 18: HLA-DRB1*07:01 - 365 
Length 19: HLA-DRB1*07:01 - 199 
Length 20: HLA-DRB1*07:01 - 330 
HLA-DRB1*11:01
Length 12: HLA-DRB1*11:01 - 54 
Length 13: HLA-DRB1*11:01 - 281 
Length 14: HLA-DRB1*11:01 - 532 

In [15]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def evaluate_models_from_peptides(peptides_dict, scores_dict, per_name_models_b, per_name_models_nb):
    results, results_scores, y_true_all = {}, {}, {}

    for allele in peptides_dict:
        results[allele] = {}
        results_scores[allele] = {}
        y_true_all[allele] = {}

        allele_formatted = allele.replace('-', '_').replace('*', '_').replace(':', '_').replace('/', '_')

        for split in peptides_dict[allele]:
            results[allele][split] = {}
            results_scores[allele][split] = {}
            y_true_all[allele][split] = {}

            split_formatted = f"s{split}"

            for length in peptides_dict[allele][split]:
                peptides = peptides_dict[allele][split][length]
                true_scores = scores_dict[allele][split][length]

                runs_b = per_name_models_b.get(allele_formatted, {}).get(split_formatted, {})
                runs_nb = per_name_models_nb.get(allele_formatted, {}).get(split_formatted, {})

                if not runs_b or not runs_nb:
                    continue

                pred_scores = calculate_scores_for_allele_split(runs_b, runs_nb, peptides)

                # Skip if length mismatch
                if len(pred_scores) != len(true_scores):
                    continue

                # Store predicted and true scores
                results_scores[allele][split][length] = pred_scores.tolist()
                y_true_all[allele][split][length] = true_scores.tolist()

                try:
                    auc = roc_auc_score(true_scores, pred_scores)
                    acc = accuracy_score(np.round(true_scores), np.round(pred_scores))
                    prec = precision_score(np.round(true_scores), np.round(pred_scores))
                    rec = recall_score(np.round(true_scores), np.round(pred_scores))
                    f1 = f1_score(np.round(true_scores), np.round(pred_scores))
                except Exception:
                    auc = acc = prec = rec = f1 = None

                results[allele][split][length] = {
                    "auc": auc,
                    "accuracy": acc,
                    "precision": prec,
                    "recall": rec,
                    "f1": f1
                }

    return results, results_scores, y_true_all


In [67]:
load_path = r"C:\Projects\grandmaster\notebooks\MHC_predictor\Atlas\atlas_cores.json"

try:
    with open(load_path, "r") as f:
        loaded_data = json.load(f)
except FileNotFoundError:
    print(f"File not found: {load_path}")
    loaded_data = None

# MixMHC

# mix of data

In [16]:
from collections import defaultdict
import numpy as np

# Initialize dictionary for peptides from loaded_data
loaded_peptides_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

# Split mapping between peptides_dict and loaded_data formats
SPLIT_MAP = {0: '1', 1: '2', 2: '3', 3: '4'}  # peptides_dict split -> loaded_data split

for allele in peptides_dict:
    # Convert allele format for loaded_data
    loaded_allele_key = allele.replace('*', '_').replace(':', '_').replace('-','_')
    
    if loaded_allele_key not in loaded_data:
        print(f"Skipping {allele} - not found in loaded_data")
        continue
    
    for split in peptides_dict[allele]:
        if split not in SPLIT_MAP:
            print(f"Skipping {allele} split {split} - no mapping to loaded_data")
            continue
            
        loaded_split = SPLIT_MAP[split]
        
        if loaded_split not in loaded_data[loaded_allele_key]:
            print(f"Skipping {allele} split {split} (mapped to {loaded_split}) - not in loaded_data")
            continue
        
        # Get all peptides (keys) and their lengths for this allele/split
        peptides_with_cores = loaded_data[loaded_allele_key][loaded_split]
        for peptide, core in peptides_with_cores.items():
            peptide_length = len(peptide)
            if peptide_length < 19:
                loaded_peptides_dict[allele][split][peptide_length].append(peptide)
        
        # Convert lists to numpy arrays to match peptides_dict structure
        for length in loaded_peptides_dict[allele][split]:
            loaded_peptides_dict[allele][split][length] = np.array(loaded_peptides_dict[allele][split][length])

# Convert defaultdict to regular dict
loaded_peptides_dict = {k: {sk: dict(sv) for sk, sv in v.items()} for k, v in loaded_peptides_dict.items()}

# Print summary
print("\nPeptides from loaded_data (length < 19):")
for allele in loaded_peptides_dict:
    print(f"{allele}:")
    for split in loaded_peptides_dict[allele]:
        lengths = loaded_peptides_dict[allele][split].keys()
        counts = [len(loaded_peptides_dict[allele][split][l]) for l in lengths]
        print(f"  Split {split}: lengths {list(lengths)} (counts: {counts})")


Peptides from loaded_data (length < 19):
HLA-DRB1*03:01:
  Split 0: lengths [15, 16, 13, 17, 12, 14, 18] (counts: [416, 443, 223, 390, 111, 350, 305])
  Split 1: lengths [14, 18, 17, 15, 16, 13, 12] (counts: [343, 279, 394, 456, 458, 228, 105])
  Split 2: lengths [14, 18, 17, 15, 16, 13, 12] (counts: [354, 272, 408, 476, 422, 191, 88])
  Split 3: lengths [16, 17, 14, 13, 18, 15, 12] (counts: [454, 396, 326, 197, 287, 456, 98])
HLA-DRB1*07:01:
  Split 0: lengths [14, 13, 17, 16, 18, 15, 12] (counts: [667, 460, 591, 737, 452, 792, 247])
  Split 1: lengths [13, 16, 17, 15, 14, 12, 18] (counts: [488, 723, 598, 762, 621, 258, 414])
  Split 2: lengths [15, 16, 17, 14, 13, 18, 12] (counts: [733, 789, 597, 670, 468, 438, 259])
  Split 3: lengths [16, 15, 13, 14, 18, 17, 12] (counts: [733, 767, 468, 652, 375, 606, 300])
HLA-DRB1*11:01:
  Split 0: lengths [16, 15, 17, 13, 12, 14, 18] (counts: [882, 988, 753, 501, 180, 767, 605])
  Split 1: lengths [15, 13, 14, 16, 17, 18, 12] (counts: [996, 415

In [18]:
import numpy as np
from collections import defaultdict

# Initialize dictionaries
filtered_peptides_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
loaded_peptides_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

# Split mapping: peptides_dict split → loaded_data split
SPLIT_MAP = {0: '1', 1: '2', 2: '3', 3: '4'}

for allele in peptides_dict:
    # Convert allele format for lookup
    loaded_allele_key = allele.replace('*', '_').replace(':', '_').replace('-', '_')
    
    if loaded_allele_key not in loaded_data:
        print(f"Skipping {allele} (no match in loaded_data)")
        continue
    
    for split in peptides_dict[allele]:
        if split not in SPLIT_MAP:
            print(f"Skipping {allele} split {split} (no mapped split in loaded_data)")
            continue
            
        loaded_split = SPLIT_MAP[split]
        
        if loaded_split not in loaded_data[loaded_allele_key]:
            print(f"Skipping {allele} split {split}→{loaded_split} (not in loaded_data)")
            continue
        
        for length in peptides_dict[allele][split]:
            if length >= 19:
                continue
                
            # Process peptides_dict (score < 0.5)
            peptides = peptides_dict[allele][split][length]
            scores = scores_dict[allele][split][length]
            mask = scores < 0.5
            filtered_peptides = peptides[mask]
            filtered_peptides_dict[allele][split][length] = filtered_peptides
            
            # Get peptides (keys) from loaded_data instead of cores (values)
            peptide_core_pairs = loaded_data[loaded_allele_key][loaded_split]
            loaded_peptides = [peptide for peptide in peptide_core_pairs.keys() if len(peptide) < 19]
            
            # Select same number as filtered peptides
            num_to_select = len(filtered_peptides)
            selected_peptides = loaded_peptides[:num_to_select]
            
            loaded_peptides_dict[allele][split][length] = np.array(selected_peptides)

# Convert to regular dicts
filtered_peptides_dict = {k: {sk: dict(sv) for sk, sv in v.items()} for k, v in filtered_peptides_dict.items()}
loaded_peptides_dict = {k: {sk: dict(sv) for sk, sv in v.items()} for k, v in loaded_peptides_dict.items()}

# Verify results
print("Results:")
print(f"Filtered peptides (score < 0.5): {sum(len(p) for a in filtered_peptides_dict.values() for s in a.values() for p in s.values())}")
print(f"Loaded peptides (matching counts): {sum(len(p) for a in loaded_peptides_dict.values() for s in a.values() for p in s.values())}")

Results:
Filtered peptides (score < 0.5): 15495
Loaded peptides (matching counts): 15495


In [19]:
filtered_peptides_dict

{'HLA-DRB1*03:01': {0: {13: array(['IDTLKKNENIKEL', 'DPMHPVTTAPSTA', 'TMLLGMLMICSAA', 'DPIELNATLSAVA',
          'PRFLEQVKHECHF', 'EWVAMTKGEGGVW', 'FTVQKGSDPKKLV', 'GEVAPDAKSFVLN',
          'TVQKGSDPKKLVL', 'VAMTKGEGGVWTF', 'WVAMTKGEGGVWT', 'RRLGARLATTGQL'],
         dtype='<U13'),
   14: array(['GWKHSVTYSCCPDT', 'SKGSSSELSAQQKK', 'WVIKESRGWKHSVT',
          'YSCCPDTPYLDITY', 'EAMSQANSAILMQR', 'EYRSTVNPWASQLG',
          'IEKFEKEAAEMGKG', 'TDRESLRNLRGYYN', 'EWVAMTKGEGGVWT',
          'FTVQKGSDPKKLVL', 'NLKWNPDDYGGVKK', 'SEHETRLVAKLFKD',
          'VAKLFKDYSSVVRP', 'WVAMTKGEGGVWTF', 'YSSVVRPVEDHRQV',
          'CFAPLYHAMDVTTQ', 'QYIKANAKFIGITE'], dtype='<U14'),
   15: array(['EDMLEVWNRVWITNN', 'MERRFTSHLPVAQRG', 'QPSKGWNDWENVPFC',
          'RGGMVAPLYGVEGTK', 'RIFGRRSIPVNEALA', 'RVWITNNPHMQDKTM',
          'TSGSPIVNRNGEVIG', 'YPSGTSGSPIVNRNG', 'AAASVPAADKFKTFE',
          'AAASWDALAAELASA', 'AAGGWDSLAAELATT', 'ADYLRMWIQAATVMS',
          'ALFHEVAKLDVVKLL', 'ANATVYMIDSVLMPP', 'APEVKYTVF

In [22]:
for experiment in experiments:
    # read binders
    hmm_params = training_parameters.ExperimentParams(experiment_name="reordered_models")
    hmm_params.model_training_params = training_parameters.SimpleModelClassIIParamsMixMHC()
    hmm_params.experiment_result_data_path = f'experiment_results/{experiment}/'
    hmm_params.data_scenario_params.splits_to_read = 4


    binders_subdir = f"{base_dir}/{experiment}/binders"
    parsed_alleles = os.listdir(binders_subdir)
    per_allele_per_split_per_run_model_binders, per_allele_per_split_per_run_history_binders = read_all_models(binders_subdir)
    print(per_allele_per_split_per_run_model_binders.keys())
    print(per_allele_per_split_per_run_model_binders[list(per_allele_per_split_per_run_model_binders.keys())[0]].keys())

    

    # read nonbinders
    base_dir_nb = r"experiment_results\reordered_models_nonbinders"
    nonbinders_subdir = f"{base_dir_nb}/{experiment}/nonbinders"
    parsed_alleles = os.listdir(nonbinders_subdir)
    per_allele_per_split_per_run_model_nonbinders, per_allele_per_split_per_run_history_nonbinders = read_all_models(nonbinders_subdir)

    if experiment.startswith("base"):
        random_subdir = f"{base_dir}/base_model_r"
        
    else: 
        random_subdir = f"{base_dir}/second_model_r"
    
    per_allele_per_split_per_run_model_r = read_all_models(random_subdir, False)


    results_metrics, results_scores, y_true_all_base  = evaluate_models(loaded_peptides_dict, filtered_peptides_dict, 
                                                per_allele_per_split_per_run_model_binders, 
                                                per_allele_per_split_per_run_model_nonbinders)
    
    results_metrics_r, results_scores_r, y_true_all_base_r  = evaluate_models(loaded_peptides_dict, filtered_peptides_dict, 
                                            per_allele_per_split_per_run_model_binders, 
                                            per_allele_per_split_per_run_model_r)


    target_path = f"{base_dir}/{experiment}/combined_data_evaluation"
    save_json(results_metrics, "metrics.json", target_path)
    save_json(results_scores, "result_scores.json", target_path)
    save_json(y_true_all_base, "y_true.json", target_path)
    save_json(results_metrics_r, "metrics_r.json", target_path)
    save_json(results_scores_r, "result_scores_r.json", target_path)
    save_json(y_true_all_base_r, "y_true_r.json", target_path)

Found ['HLA_DRB1_03_01', 'HLA_DRB1_07_01', 'HLA_DRB1_11_01', 'HLA_DRB1_12_01', 'HLA_DRB1_15_01', 'HLA_DRB3_01_01', 'HLA_DRB3_02_02', 'HLA_DRB4_01_01']
dict_keys(['HLA_DRB1_03_01', 'HLA_DRB1_07_01', 'HLA_DRB1_11_01', 'HLA_DRB1_12_01', 'HLA_DRB1_15_01', 'HLA_DRB3_01_01', 'HLA_DRB3_02_02', 'HLA_DRB4_01_01'])
dict_keys(['s0', 's1', 's2', 's3'])
Found ['HLA_DRB1_03_01', 'HLA_DRB1_07_01', 'HLA_DRB1_11_01', 'HLA_DRB1_12_01', 'HLA_DRB1_15_01', 'HLA_DRB3_01_01', 'HLA_DRB3_02_02', 'HLA_DRB4_01_01']
Found ['HLA_DRB1_03_01', 'HLA_DRB1_07_01', 'HLA_DRB1_11_01', 'HLA_DRB1_12_01', 'HLA_DRB1_15_01', 'HLA_DRB3_01_01', 'HLA_DRB3_02_02', 'HLA_DRB4_01_01']


# mixMHC

In [104]:
def prepare_split_data_separeted_length(per_length_data, per_length_weights, per_length_test_data, target_lengths):
    binders_array = np.array([per_length_data[length][i] for length in target_lengths
                              for i in range(len(per_length_data[length]))], dtype=object)
    weights_array = np.array([per_length_weights[length][i] for length in target_lengths
                               for i in range(len(per_length_weights[length]))], dtype=object)
    binders_test_array = np.array([per_length_test_data[length][i] for length in target_lengths
                                   for i in range(len(per_length_test_data[length]))], dtype=object)
    return binders_array, weights_array, binders_test_array



def MixMHC2pred_for_allele(allele, input_file, output_file):
# Construct the command
    mixmhc2pred_path = "C:\\Tools\\MixMHC2pred-2.0\\MixMHC2pred.exe"
    
    command = [
        mixmhc2pred_path,
        "-i", input_file,
        "-o", output_file,
        "-a", allele,
        "--no_context"  
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
        print(f"Prediction completed! Results saved in {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred: {e}")

def is_binder_mhc_pred(file_path, allele, target_path, ind, threshold = 5.0):
    def read_mixmhc2pred_file(file_path):
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Find the first line that doesn't start with '#' (the header of the table)
        start_line = next(i for i, line in enumerate(lines) if not line.startswith('#'))
        
        # Use pandas to read the table starting from that line
        df = pd.read_csv(file_path, skiprows=start_line, delimiter='\t')
        return df

    # Load the data
    df = read_mixmhc2pred_file(file_path)


    # Classify binders and non-binders based on %Rank columns
    df['Binder'] = df.apply(
        lambda row: '1' if (row[f"%Rank_{allele}"] < threshold) else '0',
        axis=1
    )

    # Save the results to a new CSV file
    output_file = target_path + f'\\classified_peptides_{allele}-{ind}.csv'
    df.to_csv(output_file, index=False)
    print(f"Prediction completed! Results if b or not of {allele} saved in {output_file}")
    return df



def evaluate_with_mixmhc(test_data_b, test_data_nb, target_path_allele, allele, threshold=5.0):
    """
    This function will:
    1. Use MixMHC to predict binder/non-binder for peptides.
    2. Compare predicted results with actual (test_data).
    3. Calculate performance metrics: accuracy, precision, recall, F1, and AUC.

    Parameters:
        test_data_b: Dictionary with splits as keys, each containing an array of binder peptides.
        test_data_nb: Dictionary with splits as keys, each containing an array of non-binder peptides.
        target_path_allele: Directory path to save the test file and MixMHC output.
        allele: The allele to predict results for.
        fold_idx: Index of the fold (to handle multiple splits).
        threshold: The threshold for classifying binders based on %Rank.

    Returns:
        metrics: Dictionary containing calculated metrics for the test data.
    """
    
    # Format the allele as required by MixMHC

    """allele_formatted = allele.replace("HLA-", "").replace("-", "")  # Remove 'HLA-' and '-'
    parts = allele_formatted.split('DRB')
    formatted_allele = f"DRB{parts[1][0:1]}_{parts[1][1:3]}_{parts[1][3:]}"
    """
    formatted_allele = allele.replace("HLA-", "").replace(":", "_").replace("*", "_")
    metrics = {}
    y_scores_mixmhc_all = {}
    y_true_all = {}
    # Extract binder and non-binder peptides for the current fold
    for fold_idx in test_data_b.keys():
        metrics[fold_idx] = {}
        y_scores_mixmhc_all[fold_idx] = []
        y_true_all[fold_idx] = []


        binders = test_data_b[fold_idx]  # Get binder peptides for the current fold
        non_binders = test_data_nb[fold_idx]  # Get non-binder peptides for the current fold
        binders_filtered = []
        non_binders_filtered = []
        # Filter peptides with length >= min_peptide_length
        for length in binders.keys(): 
            if length >=12:
                binders_filtered.extend(binders.get(length))
                non_binders_filtered.extend(non_binders.get(length))
            
        # Combine the binders and non-binders into a single DataFrame for test data
        test_data = pd.DataFrame({
            "peptide": list(binders_filtered) + list(non_binders_filtered),
            "true_label": [1] * len(binders_filtered) + [0] * len(non_binders_filtered)  # 1 for binder, 0 for non-binder
        })

    
        # Save the combined test peptides to a text file for MixMHC
        if not os.path.exists(target_path_allele):
                        os.makedirs(target_path_allele)

        test_file = os.path.join(target_path_allele, f"test_peptides_{formatted_allele}_{fold_idx}.txt")
        test_data["peptide"].to_csv(test_file, index=False, header=False)
        
        # Output file for MixMHC results
        output_file = os.path.join(target_path_allele, f"mixmhc2pred_results_{formatted_allele}_{fold_idx}.txt")
        
        # Run MixMHC2pred for this allele
        MixMHC2pred_for_allele(formatted_allele, test_file, output_file)
        
        if not os.path.exists(output_file):
            raise FileNotFoundError(f"MixMHC2pred output file not found: {output_file}")
        
        # Get predictions (binders and non-binders)
        df_pred = is_binder_mhc_pred(output_file, formatted_allele, target_path_allele, fold_idx, threshold)
        
        # Extract true labels
        y_true = test_data["true_label"].tolist()  # true labels (1 for binder, 0 for non-binder)
        
        # Extract predicted scores (percentage ranks)
        y_scores_mixmhc = (100 - df_pred[f"%Rank_{formatted_allele}"]).tolist()
        
        auc_score_mixmhc = roc_auc_score(y_true, y_scores_mixmhc)
        accuracy_mixmhc = accuracy_score(y_true, [1 if s > 50 else 0 for s in y_scores_mixmhc])
        precision_mixmhc = precision_score(y_true, [1 if s > 50 else 0 for s in y_scores_mixmhc], zero_division=0)
        recall_mixmhc = recall_score(y_true, [1 if s > 50 else 0 for s in y_scores_mixmhc], zero_division=0)
        f1_mixmhc = f1_score(y_true, [1 if s > 50 else 0 for s in y_scores_mixmhc], zero_division=0)
        # Calculate metrics
        metrics[fold_idx] = {
            "accuracy": accuracy_mixmhc,
            "precision": precision_mixmhc,
            "recall": recall_mixmhc,
            "f1": f1_mixmhc,
            "auc": roc_auc_score(y_true, y_scores_mixmhc)
        }
        y_scores_mixmhc_all[fold_idx] = y_scores_mixmhc
        y_true_all[fold_idx] = y_true

    
    return y_scores_mixmhc_all, y_true_all, metrics

def evaluate_all_alleles(test_data_b, test_data_nb, target_path_allele, threshold=5.0):
    """
    Evaluate multiple alleles and store metrics for each.

    Parameters:
        test_data_b: Dictionary where keys are alleles and values are dictionaries of splits (each with an array of binder peptides).
        test_data_nb: Dictionary where keys are alleles and values are dictionaries of splits (each with an array of non-binder peptides).
        target_path_allele: Directory path to save the test file and MixMHC output.
        alleles: List of alleles to evaluate.
        threshold: The threshold for classifying binders based on %Rank.

    Returns:
        all_metrics: Dictionary containing the metrics for each allele.
    """
    
    all_metrics = {}
    all_scores = {} 
    all_true_y = {}
    alleles = test_data_b.keys()
    for allele in test_data_b.keys():
        print(f"Evaluating allele: {allele}")
        
        # Get the binder and non-binder test data for this allele
        test_data_b_allele = test_data_b[allele]
        test_data_nb_allele = test_data_nb[allele]

        # Evaluate using MixMHC for the given fold
        y_scores_mixmhc_all, y_true_all, metrics = evaluate_with_mixmhc(test_data_b_allele, test_data_nb_allele, target_path_allele, allele, threshold)
        
        # Store the results
        all_metrics[allele] = metrics
        all_scores[allele] = y_scores_mixmhc_all
        all_true_y[allele] = y_true_all
    
    return all_metrics, all_true_y, all_scores

def save_json(data, filename, target_path):
    path = os.path.join(target_path, filename)
    if not os.path.exists(target_path):
        os.makedirs(target_path)
    with open(path, 'w') as file:
        json.dump(data, file, indent=4)

#preapare data to test 
hmm_params = training_parameters.ExperimentParams(experiment_name="mix_mhx")
hmm_params.model_training_params = training_parameters.SimpleModelClassIIParamsMixMHC()
hmm_params.data_scenario_params.input_data_path = r'C:\Projects\grandmaster\notebooks\alleles_data\simple_model_enrichment\per_length_per_kfold_split'
hmm_params.data_scenario_params.splits_to_read = 4
model_training_params = hmm_params.model_training_params

model_training_params.lengths_to_use = [12, 13, 14, 15, 16, 17, 18, 19, 20]
df = additional_data[0][list(additional_data[0].keys())[0]]

current_mix = ['HLA-DRB1*03:01', 'HLA-DRB1*07:01', 
               'HLA-DRB1*12:01', 'HLA-DRB1*11:01', 
               'HLA-DRB1*15:01', 'HLA-DRB3*01:01', 
               'HLA-DRB3*02:02', 'HLA-DRB4*01:01']
model_training_params: training_parameters.ModelTrainingParams = hmm_params.model_training_params
model_training_params.alleles_to_use = [ item for item in current_mix]






target_path_allele = r'C:\Projects\grandmaster\notebooks\MHC_predictor\experiments\core_identification_simple_model_enrichment\mixmhc_test\evaluation'
metrics_mixmhc, all_true_y_for_mhc, scores_mhc = evaluate_all_alleles(loaded_peptides_dict, filtered_peptides_dict, target_path_allele)



target_path = r"C:\Projects\grandmaster\notebooks\MHC_predictor\metrics_and_scores\mixMHC"
save_json(metrics_mixmhc, "metrics.json", target_path)
save_json(scores_mhc, "result_scores.json", target_path)
save_json(all_true_y_for_mhc, "y_true.json", target_path)

Evaluating allele: HLA-DRB1*03:01
Prediction completed! Results saved in C:\Projects\grandmaster\notebooks\MHC_predictor\experiments\core_identification_simple_model_enrichment\mixmhc_test\evaluation\mixmhc2pred_results_DRB1_03_01_0.txt
Prediction completed! Results if b or not of DRB1_03_01 saved in C:\Projects\grandmaster\notebooks\MHC_predictor\experiments\core_identification_simple_model_enrichment\mixmhc_test\evaluation\classified_peptides_DRB1_03_01-0.csv
Prediction completed! Results saved in C:\Projects\grandmaster\notebooks\MHC_predictor\experiments\core_identification_simple_model_enrichment\mixmhc_test\evaluation\mixmhc2pred_results_DRB1_03_01_1.txt
Prediction completed! Results if b or not of DRB1_03_01 saved in C:\Projects\grandmaster\notebooks\MHC_predictor\experiments\core_identification_simple_model_enrichment\mixmhc_test\evaluation\classified_peptides_DRB1_03_01-1.csv
Prediction completed! Results saved in C:\Projects\grandmaster\notebooks\MHC_predictor\experiments\cor

# Net MHC

In [23]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import json
import re

# Paths and data
netmhc_output_folder = r'\\wsl.localhost\Ubuntu\home\rpole\netMHCIIpan-4.3\Evaluation\netmhcpan_outputs'
metrics_netmhc = {}

# loaded_peptides_dict and filtered_peptides_dict assumed to be available
# Structure: {allele: {split: {length: [peptides]}}}

def parse_allele_and_split(filename):
    match = re.match(r'test_peptides_(.*)_([0-9]+)\.xls', filename)
    if match:
        allele_raw, split = match.groups()
        allele = "HLA-" + allele_raw.replace("_", ":")  # DRB1_03_01 -> HLA-DRB1:03:01
        return allele, int(split)
    return None, None

def extract_scores_and_labels(filepath, n_binders):
    # Read with second row as header (first row is typically a comment)
    df = pd.read_csv(filepath, sep="\t", header=1)
    
    if 'Score' not in df.columns:
        raise ValueError(f"'Score' column not found in {filepath}")
    
    scores = df['Score'].astype(float)
    y_true = [1]*n_binders + [0]*(len(scores) - n_binders)
    #print(len(scores), n_binders)
    return scores, y_true

for filename in os.listdir(netmhc_output_folder):
    if not filename.endswith(".xls"):
        continue

    allele, split = parse_allele_and_split(filename)
    if allele is None or split is None:
        continue

    filepath = os.path.join(netmhc_output_folder, filename)

    # Count number of binders
    binders = []
    for length_peps in loaded_peptides_dict.get(allele, {}).get(split, {}).values():
        binders.extend(length_peps)
        
    n_binders = len(binders)

    try:
        scores, y_true = extract_scores_and_labels(filepath, n_binders)
        scores = np.array(scores)
        y_pred = (scores >= 0.5).astype(int)  # Binary prediction based on Score threshold (adjust if needed)

        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred),
            'recall': recall_score(y_true, y_pred),
            'f1': f1_score(y_true, y_pred),
            'auc': roc_auc_score(y_true, scores)
        }

        if allele not in metrics_netmhc:
            metrics_netmhc[allele] = {}
        metrics_netmhc[allele][split] = metrics

    except Exception as e:
        print(f"Error processing {filename}: {e}")




c:\Tools\anaconda3\envs\pomegranate\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Tools\anaconda3\envs\pomegranate\lib\site-packages\sklearn\metrics\_ranking.py:375: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
c:\Tools\anaconda3\envs\pomegranate\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Tools\anaconda3\envs\pomegranate\lib\site-packages\sklearn\metrics\_ranking.py:375: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score 

In [ ]:
# Save metrics to JSON
output_json_path = "metrics_netmhc.json"
with open(output_json_path, "w") as f:
    json.dump(metrics_netmhc, f, indent=2)

print(f"Saved metrics to {output_json_path}")

In [120]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def compute_metrics(y_true, y_pred, y_scores=None):
    metrics = {}
    if len(set(y_true)) < 2:
        # Only one class present
        metrics['accuracy'] = accuracy_score(y_true, y_pred)
        metrics['precision'] = precision_score(y_true, y_pred, zero_division=0)
        metrics['recall'] = 0.0
        metrics['f1'] = 0.0
        metrics['auc'] = None  # Not computable
    else:
        metrics['accuracy'] = accuracy_score(y_true, y_pred)
        metrics['precision'] = precision_score(y_true, y_pred, zero_division=0)
        metrics['recall'] = recall_score(y_true, y_pred, zero_division=0)
        metrics['f1'] = f1_score(y_true, y_pred, zero_division=0)
        if y_scores is not None:
            metrics['auc'] = roc_auc_score(y_true, y_scores)
        else:
            metrics['auc'] = None
    return metrics


In [129]:
import os
import json
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

folder = r"\\wsl.localhost\Ubuntu\home\rpole\netMHCIIpan-4.3\Evaluation\netmhcpan_outputs"
metrics_netmhc = {}

def parse_filename(filename):
    # Example: test_peptides_DRB1_12_01_2.xls
    parts = filename.replace('.xls', '').split('_')
    split = int(parts[-1])
    allele_parts = parts[2:-1]  # ['DRB1', '12', '01']
    # Join with '*' between gene and first number, then ':' between numbers
    allele = f"HLA-{allele_parts[0]}*{int(allele_parts[1]):02d}:{int(allele_parts[2]):02d}"
    return allele, split


for file in os.listdir(folder):
    if not file.endswith('.xls'):
        continue
    filepath = os.path.join(folder, file)
    allele, split = parse_filename(file)

    # Read with header on second row (index=1)
    df = pd.read_csv(filepath, sep='\t', header=1)
    df.columns = df.columns.str.strip()  # Clean column names

    # Count number of binders and nonbinders
    binders = loaded_peptides_dict[allele][split]
    nonbinders = filtered_peptides_dict[allele][split]
    num_binders = sum(len(peps) for peps in binders.values())
    num_nonbinders = sum(len(peps) for peps in nonbinders.values())

    # True labels: 1 for binders, 0 for nonbinders
    y_true = [1]*num_binders + [0]*num_nonbinders

    # Use the "Score" column for predictions
    if "Score" not in df.columns:
        raise ValueError(f"No 'Score' column found in {file}")

    df = df.iloc[:num_binders + num_nonbinders]  # Trim to correct length
    y_scores = df["Score"].astype(float)

    # Predict binders with threshold = 0.5 (adjust if needed)
    y_pred = [1 if s >= 0.5 else 0 for s in y_scores]

    # Calculate metrics
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'auc': roc_auc_score(y_true, y_scores)
    }

    # Save results
    if allele not in metrics_netmhc:
        metrics_netmhc[allele] = {}
    metrics_netmhc[allele][split] = metrics

# Save all metrics to JSON
with open("metrics_netmhc.json", "w") as f:
    json.dump(metrics_netmhc, f, indent=2)

print("Metrics saved to metrics_netmhc.json")


Metrics saved to metrics_netmhc.json


In [131]:
import collections

# After you fill metrics_netmhc dict...

# Sort the splits dict by key (split number) for each allele
metrics_netmhc_sorted = {}
for allele, splits in metrics_netmhc.items():
    # Create an OrderedDict sorted by split key (int)
    sorted_splits = dict(sorted(splits.items(), key=lambda item: item[0]))
    metrics_netmhc_sorted[allele] = sorted_splits

# Save sorted dict to JSON
with open("metrics_netmhc.json", "w") as f:
    json.dump(metrics_netmhc_sorted, f, indent=2)


In [ ]:
# Save to JSON
output_json_path = 'metrics_netmhc.json'
with open(output_json_path, 'w') as f:
    json.dump(metrics_netmhc, f, indent=4)

print(f"Metrics saved to {output_json_path}")